**@juliaroquette 19 Nov 2023** Implementing a code for getting extinction estimates on the JHK colour-colour diagram by de-reddening YSOs down to the CTTS Loucs. 

In [11]:
import numpy as np

<u>**Reddening**</u>: The reddening law adopted is the one by [Indebetouw et al. 2005](https://ui.adsabs.harvard.edu/abs/2005ApJ...619..931I/abstract)

For Orion: l=209.0117 b=-19.3829. Nevertheless, for future simplification, I will simply adopt the averafe value from Table 1 from Indebetouw et al. 2005::

- $\frac{A_J}{A_K}=2.5\pm0.14$
- $\frac{A_H}{A_K}=1.55\pm0.08$
- $\frac{A_K}{A_K}=1$

Following Indebetouw et al. 2005 we adopt $\frac{A_K}{A_V}\sim8.8$ for $R_V=3.1$ from Cardeli+1989. Hence:

- $\frac{A_J}{A_V}=0.284\pm0.14$
- $\frac{A_H}{A_V}=1.76\pm0.08$
- $\frac{A_K}{A_K}=0.114$

In [57]:
def A_J(Av, value=None):
    if value == 'up':
        return (0.284 + 0.14)*Av
    elif value == 'bottom':
        return (0.284 - 0.14)*Av
    else:
        return 0.284*Av

def A_H(Av, value=None):
    if value == 'up':
        return (0.176 + 0.08)*Av
    elif value == 'bottom':
        return (0.176 - 0.08)*Av
    else:
        return 0.176*Av
def A_Ks(Av):
    return 0.114*Av

<u>**The Classical T Tauri Locus**</u>
The Classical T Tauri Locus has been introduced by [Mayer et al. 1997](https://ui.adsabs.harvard.edu/abs/1997AJ....114..288M/abstract) as a narrow locci occupied by de-reddened Classical T Tauri stars in the JHK Colour-colour diagram. 

$(J-H)_\mathrm{CTTS} = (0.58 \pm 0.11) \times (H-K)_\mathrm{CTTS} + (0.52 \pm 0.06)$

The CTTS locus intercepts the reddening vector of an M6 stars with Av=1.6, it extends until (H-K) = 1 mag and it was estimated based on the CIT photometric system. 

The conversion between CIT and 2MASS is provided in [Carpenter 2001](https://ui.adsabs.harvard.edu/abs/2001AJ....121.2851C/abstract):

$(Ks)_{2MASS} = K_{CIT} + (0.000 \pm 0.005)(J-K)_{CIT} + (-0.024\pm 0.003)$

$(J-H)_{2MASS} = (1.076\pm 0.010)(J-H)_{CIT} + (-0.043 \pm 0.006)$

$(J-Ks)_{2MASS} = (1.056 \pm 0.006)(J-K)_{CIT} + (-0.013 \pm 0.005)$

$(H-Ks)_{2MASS}= (1.026 \pm 0.020)(H-K)_{CIT} + (0.028 \pm 0.005)$.



<u>CTTS Locus for 2MASS:</u>
By propagating the coefficients in these transformations along with their uncertainties we obtain:
    
$(J-H)_{2M} = (0.608\pm 0.116)[(H-Ks)_{2M} - (0.028 \pm 0.005)] + (0.603 \pm 0.065)$

$(J-H)_{2M,up} = 0.724[(H-Ks)_{2M,up} - 0.033] + 0.668$
$(J-H)_{2M,bottom} = 0.492[(H-Ks)_{2M,bottom} - 0.023] + 0.538$

<u>CTTS Locus Limits:</u>

- From the [updated Pecault&Mamajek 2013 Empirical Sequence](https://www.pas.rochester.edu/~emamajek/EEM_dwarf_UBVIJHK_colors_Teff.txt) a *M6 dwarf* has the colours:

(J-H)_{2M} = 0.605

(H-Ks)_{2M} = 0.352



In [36]:
np.sqrt((1.076*0.06)**2+(.52*.01)**2+(0.006)**2)

0.06504639574949561

In [38]:
0.52*1.076-0.043 - 0.06504639574949561

0.45147360425050437

In [50]:
    def JH_ctts_2M(HK, value=None):
        if value == 'up':
            return 0.724*(HK - 0.033) + 0.538 
        elif value == 'bottom':
            return 0.492*(HK - 0.023) + 0.452
        else:
            return 0.608*(HK - 0.028) + 0.517 
    

In [52]:
JH_ctts_2M(0.352, value='up')

0.768956

In [42]:
def JH_ctts_2M(HM, value=None):
    """
    Returns extremety points for the CTTS Locus for the 2MASS system
    """
    def JH_ctts_2M(value=None):
        if value == 'up':
            return 0.724*(HK - 0.033) + 0.668 
        elif value == 'bottom':
            return 0.492*(HK - 0.023) + 0.538
        else:
            return 0.608*(HK - 0.028) + 0.603 
    

In [1]:
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

In [2]:
def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x, y
    else:
        return False

In [ ]:
def intersecao(X, Y, p0, p1):
    """
    acha interseção entre a curva da isocrona, vetores X e Y, e a reta que define o avermelhamento de um 
    ponto, descrita pelos pontos p0 (ponto de dados reais) e p1 (ponto avermelhado).
    retorna os indices ix onde as curvas se interceptam
    """
    
    A=np.array(np.transpose([X,Y]))
    #SOLUCAO DO STACK OVERFLOW PRA INTERSECAO ENTRE UMA RETA E UMA CURVA
    b = (p1[1] - p0[1]) / (p1[0] - p0[0]) # gradient
    a = p0[1] - b * p0[0] # intercept
    B = (a + A[:,0] * b) - A[:,1] # distance of y value from line
    ix = np.where(B[1:] * B[:-1] < 0)[0] # index of points where the next point is on the other side of the line
    d_ratio = B[ix] / (B[ix] - B[ix + 1]) # similar triangles work out crossing points
    cross_points = np.zeros((len(ix), 2)) # empty array for crossing points
    cross_points[:,0] = A[ix,0] + d_ratio * (A[ix+1,0] - A[ix,0]) # x crossings
    cross_points[:,1] = A[ix,1] + d_ratio * (A[ix+1,1] - A[ix,1]) # y crossings
    return ix     

In [3]:
def reta(p1, p2):
    """
    Given two points p1=[x1,y1]
    and p2=[x2,y2], estimates the A and B
    coeficients in a straight line given:
    y=Ax+B
    """
    x=np.empty([2])
    y=np.empty([2])
    x[0]=p1[0]
    x[1]=p2[0]
    y[0]=p1[1]
    y[1]=p2[1]
    A,B=np.polyfit(x,y,1)
    return A,B    

In [4]:
def avp0(p0,ax,ay):
    """
    avermelha o ponto p0 dado os avermelhamentos ax e ay 
    retorna p1
    """
    return [p0[0]-50.*ax,p0[1]-50.*ay]

In [5]:
def getAv(x,y,xi,yi,Ax,Ay,plot=False):
    """
    Estimate Avs:
    x,y: datapoints
    xi,yi: isochrone data
    Ax,Ay gives a reference for the reddening in each axis of the color-space 
    should be something like Ax=C*Av, with Av=1
    """
    from scipy import spatial
    Av=np.nan
    flag=np.nan
    #testa se esta numa regiao desavermelhavel:
    p1=[xi[np.argmax(xi)],yi[np.argmax(xi)]]
    p2=me.avp0(p1,-Ax,-Ay)
    p4=[xi[np.argmin(xi)],yi[np.argmin(xi)]]
    p3=me.avp0(p4,-Ax,-Ay)
    if bool(plot):
        fig,ax = plt.subplots(1,1,figsize = (3,3),dpi=100)
        ax.plot([p1[0],p2[0]],[p1[1],p2[1]],'g:')
        ax.plot([p2[0],p3[0]],[p2[1],p3[1]],'g:')
        ax.plot([p3[0],p4[0]],[p3[1],p4[1]],'g:')
        ax.plot([p4[0],p1[0]],[p4[1],p1[1]],'g:')
        ax.plot(x,y,'rx')
        ax.plot(xi,yi,'k-',lw=1)
        t=me.avp0([x,y],Ax,Ay)
        ax.plot([x,t[0]],[y,t[1]],'y:')

        ax.set_ylim(10,23) 
        ax.invert_yaxis()
        ax.set_xlim(0,5)
                   
    if bool(me.IsidePolygon([x,y],p1,p2,p3,p4)):
        print('here')
        ii=me.intersecao(xi,yi,[x,y],me.avp0([x,y],Ax,Ay)) #intersecao entre linha de Av e a isocrona
        lineAV=me.line([x,y],me.avp0([x,y],Ax,Ay))
#        ax.plot(x,y,'kx',ms=1)
        if bool(any(ii))==True: #testa se tem resultado]
            if len(ii)>1:
                print('here2')
#                print('degenerated! I am dealing with it....')
                Avp=[]
                for xx in ii:
                    xh=xi[xx]
                    yh=yi[xx]
                    A=np.array(np.transpose([xi,yi]))
                    pt=[xh,yh]
                    #acha ponto real
                    d,jj=spatial.KDTree(A).query(pt)
                    if (jj <len(xi) ):
                        if (jj > 0 ):
                            lineIsoc=me.line([xi[jj-1],yi[jj-1]],[xi[jj+1],yi[jj+1]])
                            x0,y0=me.intersection(lineAV,lineIsoc) 
                        #    print(x,x0,y0)
#                            print((x-x0)/Ax)
                            flag=1
                        else:
                            lineIsoc=me.line([xi[jj],yi[jj]],[xi[jj+1],yi[jj+1]])
                            x0,y0=xh,yh
                            flag=2
                    else:
                        lineIsoc=me.line([xi[jj-1],yi[jj-1]],[xi[jj],yi[jj]])
                        x0,y0=xh,yh
                        flag=3
                    Avx=(x-x0)/Ax
                    Avy=(y-y0)/Ay
 #                   print((Avx+Avy)/2.)
                    Avp.append((Avx+Avy)/2.)
#                print(Avp)
                Av=Avp[np.argmin(Avp)]
                
            else: # if there is only one intersection
                xh=xi[ii[0]]
                yh=yi[ii[0]]
                A=np.array(np.transpose([xi,yi]))
                pt=[xh,yh]
                #acha ponto real
                d,jj=spatial.KDTree(A).query(pt)
                if (jj <len(xi) ):
                    if (jj > 0 ):
                        lineIsoc=me.line([xi[jj-1],yi[jj-1]],[xi[jj+1],yi[jj+1]])
                        x0,y0=me.intersection(lineAV,lineIsoc) 
               #         print('2:',x,x0,y0)
                    else:
                        lineIsoc=me.line([xi[jj],yi[jj]],[xi[jj+1],yi[jj+1]])
                        x0,y0=xh,yh
                else:
                    lineIsoc=me.line([xi[jj-1],yi[jj-1]],[xi[jj],yi[jj]])
                    x0,y0=xh,yh
                Avx=(x-x0)/Ax
                Avy=(y-y0)/Ay
                Av=(Avx+Avy)/2.
                flag=np.nan
    return Av,flag


In [ ]:
def dered(p0,av,ax,ay,d=None):
    """
    desavermelha um ponto num diagram qualquer, com eixos x e y podendo ser cores ou magnitudes
    p0 é o ponto x,y
    av é o avermelhamento do ponto 
    ax e ay sao os coeficientes de avermelhamento nos eixos x e y
    se for um diagrama cor-magnitude, deve-se fornecer d=log10 dist(pc)
    Cygnus OB2 d=10.8
    
    retorna p1 desavermelhada
    """
    if d is None:
        return [p0-av*ax,p0-av*ay]
    else:
        return [p0-av*ax,p0-av*ay+d]
        